멀티턴


In [ ]:
# 싱글턴 함수 정의
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def gpt_reponse(text):
    stream = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {
          "role": "system",
          "content": "너는 친절한 친구같은 AI야."
        },
        {
          "role": "user",
          "content": text
        }
      ],
      temperature=0.8,
      max_tokens=1024,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stream=True
    )
    return stream


In [ ]:
# 멀티턴 함수 정의
# 싱글턴과 차이는
# 질문을 4박5일 뉴욕 일정짜줘라고 한뒤
# 3박4일로 일정 변경해줘.
# 라고 했을때, 싱글턴은 이전질문과 연결이 안되지만 멀티턴은 연결을 할 수 있음.

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


def create_generator(history):
    chat_logs = []

    for item in history:
        if item[0] is not None: # 사용자
            message =  {
              "role": "user",
              "content": item[0]
            }
            chat_logs.append(message)            
        if item[1] is not None: # 챗봇
            message =  {
              "role": "assistant",
              "content": item[1]
            }
            chat_logs.append(message)            
    
    messages=[
        {
          "role": "system",
          "content": "당신은 친절한 어시스턴트입니다. 사용자의 질문에 친절하게 대답하세요."
        }
    ]
    messages.extend(chat_logs)
        
    gen = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      temperature=0.5,
      max_tokens=4096,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stream=True
    )
    return gen

In [8]:
# 스트림

import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        g_res = create_generator(history)
        history[-1][1] = ""
        for char in g_res:
            if char.choices[0].delta.content is not None:
                history[-1][1] += char.choices[0].delta.content
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch(server_name='0.0.0.0')

/tmp/ipykernel_100953/1650636255.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://0.0.0.0:7897
* To create a public link, set `share=True` in `launch()`.
